# evolv End-to-end Snowflake AI Hands on Lab

## Overview

This HOL (hands-on lab) explores Snowflake’s newest AI and data workflow capabilities, with a focus on Openflow for unstructured data ingestion, Cortex for AI-enablement, and Snowflake Intelligence. Participants will build an end-to-end AI-powered workflow that:
- Ingests unstructured data (in the lab, we use SharePoint)
- Enriches it with semantic views
- Delivers insights through Snowflake Intelligence (Cortex AI)

By completing this HOL, you will learn how to:
- Automate ingestion & orchestration of unstructured data with Openflow
- Build semantic views for faster, businessfriendly insights
- Use Snowflake Intelligence to query and summarize data natively in Snowflake
- Unify data, AI, and productivity into one streamlined platform

## Prerequisites

1. A [free Snowflake 30-day trial environment](https://signup.snowflake.com/) to follow along with this lab.
2. Basic knowledge of SQL, database concepts, and objects.
3. Familiarity with CSV (comma-delimited) files.

## Setup

### 1. Prepare Your Lab Environment
If you haven't already, register for a [Snowflake free 30-day trial](https://signup.snowflake.com/). This lab assumes you are using a new Snowflake account created by registering for a trial and selecting the AWS cloud provider and a US region.

The Snowflake edition (Standard, Enterprise, Business Critical, etc.) you use for this lab, does not matter.

After registering, you will receive an email with an activation link and URL for accessing your Snowflake account.

Logging into the Snowflake User Interface (UI)
Open a browser window and enter the URL of your Snowflake 30-day trial environment that was sent with your registration email. Enter the username and password that you specified during the registration.

### 2. Create Dedicated Role and Warehouse
- **Role**: `EVOLV_AI_HOL`
- **Warehouse**: `EVOLV_AI_HOL_WH` (XSMALL with auto-suspend/resume)

In [ ]:
SET hol_admin_role_name = 'EVOLV_AI_HOL_ADMIN';
SET hol_warehouse_name = 'EVOLV_AI_HOL_WH';
SET current_user_name = CURRENT_USER();

-- Switch to SECURITYADMIN role to create role
USE ROLE SECURITYADMIN;
CREATE OR REPLACE ROLE IDENTIFIER($hol_admin_role_name);

-- Grant the role to the current user
GRANT ROLE IDENTIFIER($hol_admin_role_name) TO USER IDENTIFIER($current_user_name);

-- Grant create database permission to the role
USE ROLE ACCOUNTADMIN;
GRANT CREATE DATABASE ON ACCOUNT TO ROLE IDENTIFIER($hol_admin_role_name);

-- Create a dedicated warehouse for the demo with auto-suspend/resume
CREATE OR REPLACE WAREHOUSE IDENTIFIER($hol_warehouse_name)
    WITH WAREHOUSE_SIZE = 'XSMALL'
    AUTO_SUSPEND = 300
    AUTO_RESUME = TRUE;

-- Grant usage on warehouse to admin role
GRANT USAGE ON WAREHOUSE IDENTIFIER($hol_warehouse_name) TO ROLE IDENTIFIER($hol_admin_role_name);

-- Alter current user's default role and warehouse to the ones used here
ALTER USER IDENTIFIER($current_user_name) SET DEFAULT_ROLE = $hol_admin_role_name;
ALTER USER IDENTIFIER($current_user_name) SET DEFAULT_WAREHOUSE = $hol_warehouse_name;

-- Configure your account to allow cross-region inference processing which is required to leverage certain LLMs
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US';

### 3. Data Infrastructure
- **Star Schema Design**: 13 dimension tables and 4 fact tables covering Finance, Sales, Marketing, HR
- **Salesforce CRM Integration**: 3 Salesforce tables (Accounts, Opportunities, Contacts) with 62,000+ CRM records
- **Realistic Sample Data**: 210,000+ records across all business domains with complete customer journey
- **Database**: `EVOLV_E2E_SNOWFLAKE_AI_HOL` with schema `EDW`

In [ ]:
SET hol_database_name = 'EVOLV_E2E_SNOWFLAKE_AI_HOL';

-- Switch to hol role to create objects
USE ROLE IDENTIFIER($hol_admin_role_name);

-- Create database and schema
CREATE OR REPLACE DATABASE IDENTIFIER($hol_database_name);
USE DATABASE IDENTIFIER($hol_database_name);

CREATE SCHEMA IF NOT EXISTS EDW;
USE SCHEMA EDW;

In [ ]:
-- Create API Integration for GitHub (public repository access)
USE ROLE ACCOUNTADMIN;    
CREATE OR REPLACE API INTEGRATION EVOLV_GITHUB_API_INTEGRATION
    API_PROVIDER = git_https_api
    API_ALLOWED_PREFIXES = ('https://github.com/evolvconsulting/')
    ENABLED = TRUE;

GRANT USAGE ON INTEGRATION EVOLV_GITHUB_API_INTEGRATION TO ROLE IDENTIFIER($hol_admin_role_name);

-- Create Git repository integration for the public demo repository
USE ROLE IDENTIFIER($hol_admin_role_name);
CREATE OR REPLACE GIT REPOSITORY EVOLV_AI_DEMO_REPO
    API_INTEGRATION = EVOLV_GITHUB_API_INTEGRATION
    ORIGIN = 'https://github.com/evolvconsulting/snowflake-ai-demo.git';

ALTER GIT REPOSITORY EVOLV_AI_DEMO_REPO FETCH;

In [ ]:
-- Create file format for CSV files
CREATE OR REPLACE FILE FORMAT CSV_FORMAT
    TYPE = 'CSV'
    FIELD_DELIMITER = ','
    RECORD_DELIMITER = '\n'
    SKIP_HEADER = 1
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    TRIM_SPACE = TRUE
    ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE
    ESCAPE = 'NONE'
    ESCAPE_UNENCLOSED_FIELD = '\134'
    DATE_FORMAT = 'YYYY-MM-DD'
    TIMESTAMP_FORMAT = 'YYYY-MM-DD HH24:MI:SS'
    NULL_IF = ('NULL', 'null', '', 'N/A', 'n/a');
    
-- Create internal stage for copied data files
CREATE OR REPLACE STAGE INTERNAL_DATA_STAGE
    FILE_FORMAT = CSV_FORMAT
    COMMENT = 'Internal stage for copied HOL data files'
    DIRECTORY = ( ENABLE = TRUE)
    ENCRYPTION = (   TYPE = 'SNOWFLAKE_SSE');

-- Copy all CSV files from Git repository demo_data folder to internal stage
COPY FILES
INTO @INTERNAL_DATA_STAGE/demo_data/
FROM @EVOLV_AI_DEMO_REPO/branches/main/demo_data/;

-- Copy unstructured doocuments from Git repository unstructured_docs folder to internal stage
COPY FILES
INTO @INTERNAL_DATA_STAGE/unstructured_docs/
FROM @EVOLV_AI_DEMO_REPO/branches/main/unstructured_docs/;

-- Verify files were copied
LS @INTERNAL_DATA_STAGE;

ALTER STAGE INTERNAL_DATA_STAGE REFRESH;

In [ ]:
-- ========================================================================
-- DIMENSION TABLES
-- ========================================================================

-- Product Category Dimension
CREATE OR REPLACE TABLE product_category_dim (
    category_key INT PRIMARY KEY,
    category_name VARCHAR(100) NOT NULL,
    vertical VARCHAR(50) NOT NULL
);

-- Product Dimension
CREATE OR REPLACE TABLE product_dim (
    product_key INT PRIMARY KEY,
    product_name VARCHAR(200) NOT NULL,
    category_key INT NOT NULL,
    category_name VARCHAR(100),
    vertical VARCHAR(50)
);

-- Vendor Dimension
CREATE OR REPLACE TABLE vendor_dim (
    vendor_key INT PRIMARY KEY,
    vendor_name VARCHAR(200) NOT NULL,
    vertical VARCHAR(50) NOT NULL,
    address VARCHAR(200),
    city VARCHAR(100),
    state VARCHAR(10),
    zip VARCHAR(20)
);

-- Customer Dimension
CREATE OR REPLACE TABLE customer_dim (
    customer_key INT PRIMARY KEY,
    customer_name VARCHAR(200) NOT NULL,
    industry VARCHAR(100),
    vertical VARCHAR(50),
    address VARCHAR(200),
    city VARCHAR(100),
    state VARCHAR(10),
    zip VARCHAR(20)
);

-- Account Dimension (Finance)
CREATE OR REPLACE TABLE account_dim (
    account_key INT PRIMARY KEY,
    account_name VARCHAR(100) NOT NULL,
    account_type VARCHAR(50)
);

-- Department Dimension
CREATE OR REPLACE TABLE department_dim (
    department_key INT PRIMARY KEY,
    department_name VARCHAR(100) NOT NULL
);

-- Region Dimension
CREATE OR REPLACE TABLE region_dim (
    region_key INT PRIMARY KEY,
    region_name VARCHAR(100) NOT NULL
);

-- Sales Rep Dimension
CREATE OR REPLACE TABLE sales_rep_dim (
    sales_rep_key INT PRIMARY KEY,
    rep_name VARCHAR(200) NOT NULL,
    hire_date DATE
);

-- Campaign Dimension (Marketing)
CREATE OR REPLACE TABLE campaign_dim (
    campaign_key INT PRIMARY KEY,
    campaign_name VARCHAR(300) NOT NULL,
    objective VARCHAR(100)
);

-- Channel Dimension (Marketing)
CREATE OR REPLACE TABLE channel_dim (
    channel_key INT PRIMARY KEY,
    channel_name VARCHAR(100) NOT NULL
);

-- Employee Dimension (HR)
CREATE OR REPLACE TABLE employee_dim (
    employee_key INT PRIMARY KEY,
    employee_name VARCHAR(200) NOT NULL,
    gender VARCHAR(1),
    hire_date DATE
);

-- Job Dimension (HR)
CREATE OR REPLACE TABLE job_dim (
    job_key INT PRIMARY KEY,
    job_title VARCHAR(100) NOT NULL,
    job_level INT
);

-- Location Dimension (HR)
CREATE OR REPLACE TABLE location_dim (
    location_key INT PRIMARY KEY,
    location_name VARCHAR(200) NOT NULL
);

-- ========================================================================
-- FACT TABLES
-- ========================================================================

-- Sales Fact Table
CREATE OR REPLACE TABLE sales_fact (
    sale_id INT PRIMARY KEY,
    date DATE NOT NULL,
    customer_key INT NOT NULL,
    product_key INT NOT NULL,
    sales_rep_key INT NOT NULL,
    region_key INT NOT NULL,
    vendor_key INT NOT NULL,
    amount DECIMAL(10,2) NOT NULL,
    units INT NOT NULL
);

-- Finance Transactions Fact Table
CREATE OR REPLACE TABLE finance_transactions (
    transaction_id INT PRIMARY KEY,
    date DATE NOT NULL,
    account_key INT NOT NULL,
    department_key INT NOT NULL,
    vendor_key INT NOT NULL,
    product_key INT NOT NULL,
    customer_key INT NOT NULL,
    amount DECIMAL(12,2) NOT NULL,
    approval_status VARCHAR(20) DEFAULT 'Pending',
    procurement_method VARCHAR(50),
    approver_id INT,
    approval_date DATE,
    purchase_order_number VARCHAR(50),
    contract_reference VARCHAR(100),
    CONSTRAINT fk_approver FOREIGN KEY (approver_id) REFERENCES employee_dim(employee_key)
) COMMENT = 'Financial transactions with compliance tracking. approval_status should be Approved/Pending/Rejected. procurement_method should be RFP/Quotes/Emergency/Contract';

-- Marketing Campaign Fact Table
CREATE OR REPLACE TABLE marketing_campaign_fact (
    campaign_fact_id INT PRIMARY KEY,
    date DATE NOT NULL,
    campaign_key INT NOT NULL,
    product_key INT NOT NULL,
    channel_key INT NOT NULL,
    region_key INT NOT NULL,
    spend DECIMAL(10,2) NOT NULL,
    leads_generated INT NOT NULL,
    impressions INT NOT NULL
);

-- HR Employee Fact Table
CREATE OR REPLACE TABLE hr_employee_fact (
    hr_fact_id INT PRIMARY KEY,
    date DATE NOT NULL,
    employee_key INT NOT NULL,
    department_key INT NOT NULL,
    job_key INT NOT NULL,
    location_key INT NOT NULL,
    salary DECIMAL(10,2) NOT NULL,
    attrition_flag INT NOT NULL
);

-- ========================================================================
-- SALESFORCE CRM TABLES
-- ========================================================================

-- Salesforce Accounts Table
CREATE OR REPLACE TABLE sf_accounts (
    account_id VARCHAR(20) PRIMARY KEY,
    account_name VARCHAR(200) NOT NULL,
    customer_key INT NOT NULL,
    industry VARCHAR(100),
    vertical VARCHAR(50),
    billing_street VARCHAR(200),
    billing_city VARCHAR(100),
    billing_state VARCHAR(10),
    billing_postal_code VARCHAR(20),
    account_type VARCHAR(50),
    annual_revenue DECIMAL(15,2),
    employees INT,
    created_date DATE
);

-- Salesforce Opportunities Table
CREATE OR REPLACE TABLE sf_opportunities (
    opportunity_id VARCHAR(20) PRIMARY KEY,
    sale_id INT,
    account_id VARCHAR(20) NOT NULL,
    opportunity_name VARCHAR(200) NOT NULL,
    stage_name VARCHAR(100) NOT NULL,
    amount DECIMAL(15,2) NOT NULL,
    probability DECIMAL(5,2),
    close_date DATE,
    created_date DATE,
    lead_source VARCHAR(100),
    type VARCHAR(100),
    campaign_id INT
);

-- Salesforce Contacts Table
CREATE OR REPLACE TABLE sf_contacts (
    contact_id VARCHAR(20) PRIMARY KEY,
    opportunity_id VARCHAR(20) NOT NULL,
    account_id VARCHAR(20) NOT NULL,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    email VARCHAR(200),
    phone VARCHAR(50),
    title VARCHAR(100),
    department VARCHAR(100),
    lead_source VARCHAR(100),
    campaign_no INT,
    created_date DATE
);

In [ ]:
-- ========================================================================
-- LOAD DIMENSION DATA FROM INTERNAL STAGE
-- ========================================================================

-- Load Product Category Dimension
COPY INTO product_category_dim
FROM @INTERNAL_DATA_STAGE/demo_data/product_category_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Product Dimension
COPY INTO product_dim
FROM @INTERNAL_DATA_STAGE/demo_data/product_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Vendor Dimension
COPY INTO vendor_dim
FROM @INTERNAL_DATA_STAGE/demo_data/vendor_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Customer Dimension
COPY INTO customer_dim
FROM @INTERNAL_DATA_STAGE/demo_data/customer_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Account Dimension
COPY INTO account_dim
FROM @INTERNAL_DATA_STAGE/demo_data/account_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Department Dimension
COPY INTO department_dim
FROM @INTERNAL_DATA_STAGE/demo_data/department_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Region Dimension
COPY INTO region_dim
FROM @INTERNAL_DATA_STAGE/demo_data/region_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Sales Rep Dimension
COPY INTO sales_rep_dim
FROM @INTERNAL_DATA_STAGE/demo_data/sales_rep_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Campaign Dimension
COPY INTO campaign_dim
FROM @INTERNAL_DATA_STAGE/demo_data/campaign_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Channel Dimension
COPY INTO channel_dim
FROM @INTERNAL_DATA_STAGE/demo_data/channel_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Employee Dimension
COPY INTO employee_dim
FROM @INTERNAL_DATA_STAGE/demo_data/employee_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Job Dimension
COPY INTO job_dim
FROM @INTERNAL_DATA_STAGE/demo_data/job_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Location Dimension
COPY INTO location_dim
FROM @INTERNAL_DATA_STAGE/demo_data/location_dim.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- ========================================================================
-- LOAD FACT DATA FROM INTERNAL STAGE
-- ========================================================================

-- Load Sales Fact
COPY INTO sales_fact
FROM @INTERNAL_DATA_STAGE/demo_data/sales_fact.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Finance Transactions
COPY INTO finance_transactions
FROM @INTERNAL_DATA_STAGE/demo_data/finance_transactions.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Marketing Campaign Fact
COPY INTO marketing_campaign_fact
FROM @INTERNAL_DATA_STAGE/demo_data/marketing_campaign_fact.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load HR Employee Fact
COPY INTO hr_employee_fact
FROM @INTERNAL_DATA_STAGE/demo_data/hr_employee_fact.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- ========================================================================
-- LOAD SALESFORCE DATA FROM INTERNAL STAGE
-- ========================================================================

-- Load Salesforce Accounts
COPY INTO sf_accounts
FROM @INTERNAL_DATA_STAGE/demo_data/sf_accounts.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Salesforce Opportunities
COPY INTO sf_opportunities
FROM @INTERNAL_DATA_STAGE/demo_data/sf_opportunities.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- Load Salesforce Contacts
COPY INTO sf_contacts
FROM @INTERNAL_DATA_STAGE/demo_data/sf_contacts.csv
FILE_FORMAT = CSV_FORMAT
ON_ERROR = 'CONTINUE';

-- ========================================================================
-- VERIFICATION
-- ========================================================================

-- Show all tables
SHOW TABLES IN SCHEMA EDW;

## Cortex Services & Semantic Views

### 1. Create Cortex Search Services (4 Domain-Specific)
- **Finance Documents**: Expense policies, financial reports, vendor contracts
- **HR Documents**: Employee handbook, performance guidelines, department overviews
- **Marketing Documents**: Campaign strategies, performance reports, marketing plans
- **Sales Documents**: Sales playbooks, customer success stories, performance data

#### a. Parse text from the PDF documents

In [ ]:
CREATE OR REPLACE TABLE PARSED_CONTENT AS
SELECT
    relative_path,
    CONCAT('https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/', relative_path) AS file_url,
    TO_FILE(BUILD_STAGE_FILE_URL('@EDW.INTERNAL_DATA_STAGE', relative_path)) AS file_object,
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        @EDW.INTERNAL_DATA_STAGE,
        relative_path,
        {'mode':'LAYOUT'}
        ):content::string AS content
FROM DIRECTORY(@EDW.INTERNAL_DATA_STAGE)
WHERE relative_path ILIKE 'unstructured_docs/%.pdf';

#### b. Create the Cortex Search Services over each domain-specific content

In [ ]:
-- Create search service for finance documents
-- This enables semantic search over finance-related content
CREATE OR REPLACE CORTEX SEARCH SERVICE EDW.SEARCH_FINANCE_DOCS
    ON content
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = $hol_warehouse_name
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            REGEXP_SUBSTR(relative_path, '[^/]+$') as title, -- Extract filename as title
            content
        FROM parsed_content
        WHERE relative_path ilike '%/finance/%'
    );

-- Create search service for HR documents
-- This enables semantic search over HR-related content
CREATE OR REPLACE CORTEX SEARCH SERVICE EDW.SEARCH_HR_DOCS
    ON content
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = $hol_warehouse_name
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            REGEXP_SUBSTR(relative_path, '[^/]+$') as title,
            content
        FROM parsed_content
        WHERE relative_path ilike '%/hr/%'
    );

-- Create search service for marketing documents
-- This enables semantic search over marketing-related content
CREATE OR REPLACE CORTEX SEARCH SERVICE EDW.SEARCH_MARKETING_DOCS
    ON content
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = $hol_warehouse_name
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            REGEXP_SUBSTR(relative_path, '[^/]+$') as title,
            content
        FROM parsed_content
        WHERE relative_path ilike '%/marketing/%'
    );

-- Create search service for sales documents
-- This enables semantic search over sales-related content
CREATE OR REPLACE CORTEX SEARCH SERVICE EDW.SEARCH_SALES_DOCS
    ON content
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = $hol_warehouse_name
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            REGEXP_SUBSTR(relative_path, '[^/]+$') as title,
            content
        FROM parsed_content
        WHERE relative_path ilike '%/sales/%'
    );

#### c. Snowflake Openflow Demo

Snowflake Openflow provides a scalable, no-code alternative for ingesting, parsing and indexing unstructured data. Openflow is not available within Snowflake Trial accounts and therefore has been excluding from this lab; however, let's take a moment to observe how the above two steps is accomplished with Openflow.

### 2. Create the Semantic Views (4 Business Domains)

AI-powered Business Intelligence (BI) and conversational analytics hold immense promise for data-driven decision-making. However, directly applying these technologies to complex enterprise schemas often leads to inaccurate or inconsistent results, commonly referred to as "hallucinations." This challenge arises because AI and BI systems may struggle to interpret raw data without a clear understanding of the organizational context and definitions.

Semantic Views address these challenges by capturing and storing semantic model information directly within the database, including business-relevant concepts such as metrics (e.g., total sales), dimensions (e.g., product category), and facts.

In this lab, we will be creating 4 Semantic Views:

- **Finance Semantic View**: Financial transactions, accounts, departments, vendors
- **Sales Semantic View**: Sales data, customers, products, regions, sales reps
- **Marketing Semantic View**: Campaign performance, channels, leads, impressions + **Revenue Attribution** (Salesforce CRM integration)
- **HR Semantic View**: Employee data, departments, jobs, locations, attrition

#### Basic Elements of a Semantic View Definition
Every semantic view definition requires essential elements:

- **Physical model objects**: These refer to your existing tables, views, or (in future releases) SQL queries that contain the raw data.
- **Relationships**: These define how your physical objects connect to each other (e.g., a CUSTOMER table linked to an ORDERS table).
- **Dimensions**: These are business-friendly attributes used to group or filter your data (e.g., customer's birth year, product category).
- **Metrics**: These are business-friendly calculations or aggregations, often representing Key Performance Indicators (KPIs) (e.g., total sales price, total sales quantity).

#### Best practices for creating a semantic view
- **Provide clear descriptions**:
    - Use business terminology in all names and descriptions.
    - Make descriptions detailed enough for non-technical users to understand.
- **Include representative user questions**:
    - Include questions that can help the model generator better understand your intent.
    - Include variations of how questions might be asked.
- **Review generated suggestions carefully**:
    - Make sure the questions are relevant for the use case.
    - Make sure the suggested relationships match your business understanding.
- **Test with real questions**:
    - After creating your semantic view, test it with actual business questions.
    - Refine your semantic view, based on how well the model supports these questions.
- **Iterate on developing the semantic view**:
    - Start with a simple star schema.
    - Start with core tables and metrics, then expand. We suggest three tables to keep things simple.
    - Get feedback from business users, and refine your semantic view.

#### a. Create a semantic view using Snowsight
1. Access the wizard for creating semantic views in one of the following ways:
    - Database object explorer:
        1. Sign in to Snowsight.
        2. In the navigation menu, select Catalog » Database Explorer.
        3. Select the database and schema where you want to create the semantic view.
        4. Select Create » Semantic View » Create with guided setup.

    - Cortex Analyst:
        1. Sign in to Snowsight.
        2. In the navigation menu, select AI & ML » Cortex Analyst.
        3. Select Create new » Create new Semantic View.
        
        ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/01.png)

2. In the Getting started step in the wizard:

    1. From Location to store, select the database and schema where the model should be stored: `EVOLV_E2E_SNOWFLAKE_AI_HOL.EDW`
    2. In Name, enter a name for your semantic view: `FINANCE_SEMANTIC_VIEW`
    3. Description, describe the data that your semantic view makes available: `Semantic view for financial analysis and reporting`
    4. Select **Next**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/02.png)

3. In the Select tables step in the wizard:

    1. In the All tab, select the tables or views that contain the data that you want to use in your semantic view: `ACCOUNT_DIM`, `CUSTOMER_DIM`, `DEPARTMENT_DIM`, `FINANCE_TRANSACTIONS`, `PRODUCT_DIM`, `VENDOR_DIM`
        - For better performance, don’t select more than ten tables.
        - If you want see the list of tables and views that you selected, select the Selected tab.
    2. Select **Next**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/03.png)

4. In the Select columns step in the wizard:

    1. Select the columns to include in the view.  For the purpose of this lab, select all the columns.
        - To select all columns in a table or view, select the table or view.
        - For better performance, don’t select more than 50 columns.
    2. Select **Create and Save**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/04.png)

5. The semantic view wizard completes and opens the Semantic View editor.  From here we will first focus on updating additional semantic information for the ACCOUNTS logical table:

    1. Select Edit on the ACCOUNT_DIM Logical table.
    2. Select **Generate fields** to use AI to inspect the data and populate the description and synonyms.
    3. Rename the Logical table name to: `ACCOUNTS`
    4. Select **Save**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/07.png)

6. Continue to update the semantic information for each of the Dimensions on the ACCOUNTS logical table:

    1. Under the ACCOUNTS logical table, select Dimensions to expand to show each dimension.
    2. Select Edit on the ACCOUNT_NAME dimension.
    3. Select **Generate fields** to use AI to inspect the data and populate the description and synonyms.
    4. Select **Save**.
    5. Select Edit on the ACCOUNT_TYPE dimension.
    6. Select **Generate fields** to use AI to inspect the data and populate the description and synonyms.
    7. Select **+ Sample Values** to inspect the data and automatically include sample values. Adding sample values is recommended for dimensions with relatively low-cardinality (about 1 - 10 distinct values) in which a value may be referenced by a user in a question. Before Cortex Analyst generates a SQL query for a question, it does a semantic similarity search between your question and the provided sample values to identify any appropriate literal values that may be needed to write your query.
    
    8. Select **Save**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/08.png)

7. Remove the ACCOUNT_KEY fact as this column was incorrectly added as a fact within the semantic view by the wizard.

    1. Under the ACCOUNTS logical table, select Facts to expand to show each fact.
    2. Select the elipsis on the ACCOUNT_KEY fact, select **Remove fact**, and in the confirmation dialog select **Remove**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/09.png)

8. Now we will focus on the FINANCE_TRANSACTIONS logical table. Update the additional semantic information for the FINANCE_TRANSACTIONS logical table:

    1. Select Edit on the FINANCE_TRANSACTIONS Logical table.
    2. Select **Generate fields** to use AI to inspect the data and populate the description and synonyms.
    3. Rename the Logical table name to: `TRANSACTIONS`
    4. Select **Save**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/10.png)

9. Review the Time Dimensions APPROVAL_DATE and DATE.  These are automatically categorized as Time Dimensions because of the underlying DATE data type (TIMESTAMP data types are also categorized as Time Dimensions). Cortex Analyst can handle time-based queries on Time Dimensions, Example:
    - "How many transactions occured in September 2025?"
    - "What was the total transaction amount in Q3?"

10. Update the Facts for the TRANSACTIONS logical table.  A Fact is a raw numeric field that can be aggregated.

    1. Expand the Facts section under the TRANSACTIONS logical table and remove all the facts except AMOUNT using the same steps used above in step 7.  These are created automatically based on having a numeric data type but should be removed as we will not be aggregating by these.
    2. Select Edit on the ACCOUNT fact.
    3. Select **Generate fields** to use AI to populate the description and synonyms.
    4. Select **Save**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/11.png)

11. Add Metrics for the TRANSACTIONS logical table.  A Metric is an aggregate calculation defined once for consistency.

    1. Add a new Metric by selecting the plus icon to the right of the Metrics header and enter the following values:
        - Expression: `SUM(AMOUNT)`
        - Metric name: `TOTAL_AMOUNT`
    2. Select **Generate fields** to use AI to populate the Metric description and synonyms.
    3. Select **Add**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/12.png)

    4. Add a second Metric by again selecting the plus icon to the right of the Metrics header and enter the following values:
        - Expression: `AVG(AMOUNT)`
        - Metric name: `AVERAGE_AMOUNT`
    5. Select **Generate fields** to use AI to populate the Metric description and synonyms.
    6. Select **Add**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/13.png)

    7. Add a third Metric by again selecting the plus icon to the right of the Metrics header and enter the following values:
        - Expression: `COUNT(TRANSACTION_ID)`
        - Metric name: `TOTAL_TRANSACTIONS`
    8. Select **Generate fields** to use AI to populate the Metric description and synonyms.
    9. Select **Add**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/14.png)

12. Add Relationships to define how the logical tables join. Relationships require the right column to be a primary or unique key because Cortex Analyst only generates many-to-one or one-to-one relationships. For many-to-many, consider creating a shared dimension in a view and then defining two many-to-one relationships.

    1. Select the plus icon to the right of the Relationships header to add a new relationships and and enter the following values:
        - Relationship name: `TRANSACTIONS_TO_ACCOUNTS`
        - Left table: `TRANSACTIONS`
        - Right table: `ACCOUNTS`
        - Left column(s): `ACCOUNT_KEY`
        - Right column(s): `ACCOUNT_KEY`

        Note: There is only a single option to select from in the Right column list as this is the single column defined as the Primary Key for the ACCOUNTS table. The primary key was inferred automatically when we added the ACCOUNTS table earlier. If the primary key does not show in the select list, you will need to return to edit the table information and specifcy the Primary key column.

    2. Select **Add**.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/15.png)

13. Test your Semantic View in the Cortex Analyst Playground.

    1. If the chat pane is not already displayed, select **Open chat**.
    2. Navigate to the Playground tab.
    3. Enter `What is the total COGS by quarter since 2024?` as a test prompt and select **Run**.
    4. Most likely Cortex Analyst was unable to answer this question.  Let's refine our semantic view to provide more clarity on how to filter by account.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/16.png)

    5. Return to the ACCOUNTS logical table andselect **Edit** on the ACCOUNT_NAME dimension.
    6. Replace the Dimension description with: `The name of the account within the chart of accounts used for financial transactions and accounting purposes.`
    7. Replace the Synonyms with `account`.
    8. Select **+ Sample Values** to inspect the data and automatically include sample values.  COGS should now display as a sample value.
    9. Select **Save**.
    10. Return to the **Playground** and reset the chat by selected the Refresh icon in the upper right.
    11. Rerun the earlier prompt by entering `What is the total COGS by quarter since 2024?` and select **Run**. With the refinements made to the ACCOUNT_NAME dimension, Cortex Analyst is now able to clearly interpret the question and generate the correct query.
    12. Select **+ Verified query**. Review the generated query and if determined to be accurate, select **Save and continue** to save the query. The Cortex Analyst Verified Query Repository (VQR) can help improve the accuracy and trustworthiness of results by providing a collection of questions and corresponding SQL queries to answer them. Cortex Analyst then leverages relevant SQL queries from the repository when answering similar questions.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/17.png)

We have now demonstrated the basics of setting up a semantic view within Snowsight. To complete the semantic view, we could repeat the above steps for the remaining tables. For purposes of this lab, we will script the creation of the semantic view definitions.

Note: When building your own semantic views for use with Cortex Analyst, keep in mind the following:
- Add sample queries to the Verified Queries section.
    - These are example queries that help Cortex Analyst understand how to use the semantic view.
    - Add queries that represent common use cases for your data.
    - Queries can be added directly from the Cortex Analyst Playground.
- Add synonyms for your tables, facts, dimensions, or metrics.
    - These are alternative terms that users might use in queries.
    - Synonyms help Cortex Analyst correctly interpret user questions.
- Add custom instructions.
    - These provide additional context about how the data should be interpreted.
    - Include business rules or constraints that should be considered.

#### b. Create the semantic views using SQL

In [ ]:
-- ========================================================================
-- FINANCE SEMANTIC VIEW
-- ========================================================================

create or replace semantic view EDW.FINANCE_SEMANTIC_VIEW
	tables (
		ACCOUNTS as ACCOUNT_DIM primary key (ACCOUNT_KEY) with synonyms=('chart of accounts','account types') comment='Account dimension for financial categorization',
		CUSTOMERS as CUSTOMER_DIM primary key (CUSTOMER_KEY) with synonyms=('clients') comment='Customer dimension for revenue analysis',
		DEPARTMENTS as DEPARTMENT_DIM primary key (DEPARTMENT_KEY) with synonyms=('department','business units') comment='Department dimension for cost center analysis',
		PRODUCTS as PRODUCT_DIM primary key (PRODUCT_KEY) with synonyms=('items') comment='Product dimension for transaction analysis',
		TRANSACTIONS as FINANCE_TRANSACTIONS primary key (TRANSACTION_ID) with synonyms=('finance transactions','financial data','accounting records','financial ledger','financial transactions','ledger entries') comment='All financial transactions across departments',
		VENDORS as VENDOR_DIM primary key (VENDOR_KEY) with synonyms=('suppliers') comment='Vendor information for spend analysis'
	)
	relationships (
		TRANSACTIONS_TO_ACCOUNTS as TRANSACTIONS(ACCOUNT_KEY) references ACCOUNTS(ACCOUNT_KEY),
		TRANSACTIONS_TO_CUSTOMERS as TRANSACTIONS(CUSTOMER_KEY) references CUSTOMERS(CUSTOMER_KEY),
		TRANSACTIONS_TO_DEPARTMENTS as TRANSACTIONS(DEPARTMENT_KEY) references DEPARTMENTS(DEPARTMENT_KEY),
		TRANSACTIONS_TO_PRODUCTS as TRANSACTIONS(PRODUCT_KEY) references PRODUCTS(PRODUCT_KEY),
		TRANSACTIONS_TO_VENDORS as TRANSACTIONS(VENDOR_KEY) references VENDORS(VENDOR_KEY)
	)
	facts (
		TRANSACTIONS.AMOUNT as AMOUNT with synonyms=('charge','cost','expense','figure','payment','price','quantity','sum','total','value') comment='The monetary value of the transaction, representing the amount of money either credited or debited from an account in dollars.'
	)
	dimensions (
		ACCOUNTS.ACCOUNT_NAME as ACCOUNT_NAME with synonyms=('account') comment='The name of the account within the chart of accounts used for financial transactions and accounting purposes.',
		ACCOUNTS.ACCOUNT_TYPE as ACCOUNT_TYPE with synonyms=('type','category') comment='Type of account (Income/Expense)',
		CUSTOMERS.CUSTOMER_NAME as CUSTOMER_NAME with synonyms=('customer','client','account holder','buyer') comment='The full name of the customer.',
		CUSTOMERS.INDUSTRY as INDUSTRY with synonyms=('field','market','sector','vertical') comment='The type of industry or sector in which the customer operates.',
		CUSTOMERS.STATE as STATE with synonyms=('area','location','province','region','territory') comment='The two-character code representing the state in which the customer is located.',
		DEPARTMENTS.DEPARTMENT_NAME as DEPARTMENT_NAME with synonyms=('department','business unit','organizational unit','team') comment='The full name of the department within the organization.',
		PRODUCTS.CATEGORY_NAME as CATEGORY_NAME with synonyms=('category','classification','product category','product group') comment='The category to which the product belongs, such as Electronics, Clothing, Home Goods, etc.',
		PRODUCTS.PRODUCT_NAME as PRODUCT_NAME with synonyms=('product','item','item description','product_description') comment='The name of the product.',
		TRANSACTIONS.APPROVAL_DATE as APPROVAL_DATE with synonyms=('approved date','date approved') comment='The date on which the financial transaction was approved.',
		TRANSACTIONS.APPROVAL_STATUS as APPROVAL_STATUS with synonyms=('approval','status','approval state') comment='Transaction approval status (Approved/Pending/Rejected)',
		TRANSACTIONS.CONTRACT_REFERENCE as CONTRACT_REFERENCE with synonyms=('contract','contract number','contract ref') comment='Reference to related contract',
		TRANSACTIONS.DATE as DATE with synonyms=('date','transaction date') comment='Date of the financial transaction.',
		TRANSACTIONS.PROCUREMENT_METHOD as PROCUREMENT_METHOD with synonyms=('procurement','method','purchase method') comment='Method of procurement (RFP/Quotes/Emergency/Contract)',
		TRANSACTIONS.PURCHASE_ORDER_NUMBER as PURCHASE_ORDER_NUMBER with synonyms=('PO number','PO','purchase order') comment='Purchase order number for tracking',
		VENDORS.STATE as STATE with synonyms=('area','location','province','region','territory') comment='The two-character code representing the state in which the vendor is located.',
		VENDORS.VENDOR_NAME as VENDOR_NAME with synonyms=('vendor','supplier','provider') comment='The name of the vendor or supplier that provides goods or services to the organization.',
		VENDORS.VERTICAL as VERTICAL with synonyms=('industry','market','sector') comment='The industry or market segment in which the vendor operates.'
	)
	metrics (
		TRANSACTIONS.AVERAGE_AMOUNT as AVG(AMOUNT) with synonyms=('average cost','average value','mean amount','mean value','typical amount') comment='The average amount of a transaction, calculated by dividing the total transaction amount by the number of transactions.',
		TRANSACTIONS.TOTAL_AMOUNT as SUM(AMOUNT) with synonyms=('aggregate amount','grand total','overall amount','total cost','total sum','total value') comment='The total monetary value of a single transaction, including all fees and charges.',
		TRANSACTIONS.TOTAL_TRANSACTIONS as COUNT(TRANSACTION_ID) with synonyms=('transaction count','overall transactions','total transaction volume','total transactions count') comment='The total number of transactions made which can be used to track and analyze transactional activity and behavior.'
	)
	comment='Semantic view for financial analysis and reporting'
	with extension (CA='{"tables":[{"name":"ACCOUNTS","dimensions":[{"name":"ACCOUNT_NAME","sample_values":["COGS","Revenue","Expenses"]},{"name":"ACCOUNT_TYPE","sample_values":["Expense","Income"]}]},{"name":"CUSTOMERS","dimensions":[{"name":"CUSTOMER_NAME"},{"name":"INDUSTRY","sample_values":["Retailer","Manufacturing","Tech"]},{"name":"STATE"}]},{"name":"DEPARTMENTS","dimensions":[{"name":"DEPARTMENT_NAME","sample_values":["Treasury","Accounting","Marketing"]}]},{"name":"PRODUCTS","dimensions":[{"name":"CATEGORY_NAME","sample_values":["Hardware","Machinery","Raw Materials"]},{"name":"PRODUCT_NAME"},{"name":"VERTICAL","sample_values":["Retail","Manufacturing","Tech"]}]},{"name":"TRANSACTIONS","dimensions":[{"name":"APPROVAL_STATUS"},{"name":"CONTRACT_REFERENCE"},{"name":"PROCUREMENT_METHOD"},{"name":"PURCHASE_ORDER_NUMBER"}],"facts":[{"name":"AMOUNT"}],"metrics":[{"name":"AVERAGE_AMOUNT"},{"name":"TOTAL_AMOUNT"},{"name":"TOTAL_TRANSACTIONS"}],"time_dimensions":[{"name":"APPROVAL_DATE"},{"name":"DATE"}]},{"name":"VENDORS","dimensions":[{"name":"STATE"},{"name":"VENDOR_NAME"},{"name":"VERTICAL","sample_values":["Retail","Manufacturing","Tech"]}]}],"relationships":[{"name":"TRANSACTIONS_TO_ACCOUNTS"},{"name":"TRANSACTIONS_TO_CUSTOMERS"},{"name":"TRANSACTIONS_TO_DEPARTMENTS"},{"name":"TRANSACTIONS_TO_PRODUCTS"},{"name":"TRANSACTIONS_TO_VENDORS"}],"verified_queries":[{"name":"What is the total COGS by quarter since 2024?","question":"What is the total COGS by quarter since 2024?","sql":"SELECT\\n  DATE_TRUNC(''QUARTER'', t.date) AS quarter,\\n  SUM(t.amount) AS total_cogs\\nFROM\\n  transactions AS t\\n  LEFT OUTER JOIN accounts AS a ON t.account_key = a.account_key\\nWHERE\\n  a.account_name = ''COGS''\\n  AND DATE_PART(''year'', t.date) >= 2024\\nGROUP BY\\n  DATE_TRUNC(''QUARTER'', t.date)\\nORDER BY\\n  quarter DESC NULLS LAST","use_as_onboarding_question":false,"verified_by":"Trent Foley","verified_at":1759947836}]}')
	;


-- ========================================================================
-- SALES SEMANTIC VIEW
-- ========================================================================

create or replace semantic view EDW.SALES_SEMANTIC_VIEW
	tables (
		CUSTOMERS as CUSTOMER_DIM primary key (CUSTOMER_KEY) with synonyms=('accounts','clients','customers') comment='Customer information for sales analysis',
		PRODUCTS as PRODUCT_DIM primary key (PRODUCT_KEY) with synonyms=('items','products','SKUs') comment='Product catalog for sales analysis',
		PRODUCT_CATEGORIES as PRODUCT_CATEGORY_DIM primary key (CATEGORY_KEY) with synonyms=('category','product category','product classification','product group') comment='This table stores information about product categories, including the vertical or business segment to which the category belongs, enabling analysis and reporting by category and vertical.',
		REGIONS as REGION_DIM primary key (REGION_KEY) with synonyms=('areas','regions','territories') comment='Regional information for territory analysis',
		SALES as SALES_FACT primary key (SALE_ID) with synonyms=('sales data','sales transactions') comment='All sales transactions and deals',
		SALES_REPS as SALES_REP_DIM primary key (SALES_REP_KEY) with synonyms=('reps','sales representatives','salespeople') comment='Sales representative information',
		VENDORS as VENDOR_DIM primary key (VENDOR_KEY) with synonyms=('suppliers','vendors') comment='Vendor information for supply chain analysis'
	)
	relationships (
		PRODUCT_TO_CATEGORY as PRODUCTS(CATEGORY_KEY) references PRODUCT_CATEGORIES(CATEGORY_KEY),
		SALES_TO_CUSTOMERS as SALES(CUSTOMER_KEY) references CUSTOMERS(CUSTOMER_KEY),
		SALES_TO_PRODUCTS as SALES(PRODUCT_KEY) references PRODUCTS(PRODUCT_KEY),
		SALES_TO_REGIONS as SALES(REGION_KEY) references REGIONS(REGION_KEY),
		SALES_TO_REPS as SALES(SALES_REP_KEY) references SALES_REPS(SALES_REP_KEY),
		SALES_TO_VENDORS as SALES(VENDOR_KEY) references VENDORS(VENDOR_KEY)
	)
	facts (
		SALES.SALE_AMOUNT as amount comment='Sale amount in dollars',
		SALES.UNITS_SOLD as units comment='Number of units sold'
	)
	dimensions (
		CUSTOMERS.CUSTOMER_INDUSTRY as INDUSTRY with synonyms=('customer type','industry') comment='Customer industry',
		CUSTOMERS.CUSTOMER_NAME as customer_name with synonyms=('account','client','customer') comment='Name of the customer',
		PRODUCTS.PRODUCT_NAME as product_name with synonyms=('item','product') comment='Name of the product',
		PRODUCT_CATEGORIES.CATEGORY_NAME as CATEGORY_NAME with synonyms=('category_label','category_title','classification_name','product_category_description','product_group') comment='The category to which a product belongs, such as electronics, clothing, or software as a service.',
		PRODUCT_CATEGORIES.VERTICAL as VERTICAL with synonyms=('business_area','category_group','domain','industry','market','sector') comment='The industry or sector in which a product is categorized, such as retail, technology, or manufacturing.',
		REGIONS.REGION_NAME as region_name with synonyms=('area','region','territory') comment='Name of the region',
		SALES.SALE_DATE as date with synonyms=('date','sale date','transaction date') comment='Date of the sale',
		SALES_REPS.SALES_REP_NAME as REP_NAME with synonyms=('representative','sales rep','salesperson') comment='Name of the sales representative',
		VENDORS.VENDOR_NAME as vendor_name with synonyms=('provider','supplier','vendor') comment='Name of the vendor'
	)
	metrics (
		SALES.AVERAGE_DEAL_SIZE as AVG(amount) comment='Average deal size',
		SALES.AVERAGE_UNITS_PER_SALE as AVG(units) comment='Average units per sale',
		SALES.TOTAL_DEALS as COUNT(SALE_ID) with synonyms=('count of sales','total sales') comment='Total number of deals',
		SALES.TOTAL_REVENUE as SUM(amount) with synonyms=('sales','sales revenue','total sales') comment='Total sales revenue',
		SALES.TOTAL_UNITS as SUM(units) comment='Total units sold'
	)
	comment='Semantic view for sales analysis and performance tracking'
	with extension (CA='{"tables":[{"name":"CUSTOMERS","dimensions":[{"name":"CUSTOMER_INDUSTRY","sample_values":["Retailer","Tech","Manufacturing"]},{"name":"CUSTOMER_NAME","sample_values":["Bailey and Sons","Oliver Ltd","Santos-Edwards"]}]},{"name":"PRODUCTS","dimensions":[{"name":"PRODUCT_NAME"}]},{"name":"PRODUCT_CATEGORIES","dimensions":[{"name":"CATEGORY_NAME","sample_values":["Electronics","Apparel","SaaS"]},{"name":"VERTICAL","sample_values":["Retail","Tech","Manufacturing"]}]},{"name":"REGIONS","dimensions":[{"name":"REGION_NAME","sample_values":["North","South","West"]}]},{"name":"SALES","facts":[{"name":"SALE_AMOUNT"},{"name":"UNITS_SOLD"}],"metrics":[{"name":"AVERAGE_DEAL_SIZE"},{"name":"AVERAGE_UNITS_PER_SALE"},{"name":"TOTAL_DEALS"},{"name":"TOTAL_REVENUE"},{"name":"TOTAL_UNITS"}],"time_dimensions":[{"name":"SALE_DATE","sample_values":["2022-01-01","2022-01-02","2022-01-03"]}]},{"name":"SALES_REPS","dimensions":[{"name":"SALES_REP_NAME","sample_values":["Grant Frey","Elizabeth George","Olivia Mcdaniel"]}]},{"name":"VENDORS","dimensions":[{"name":"VENDOR_NAME","sample_values":["Sullivan and Sons","Smith, Sandoval and Parker","Moore, French and Moore"]}]}],"relationships":[{"name":"PRODUCT_TO_CATEGORY"},{"name":"SALES_TO_CUSTOMERS","relationship_type":"many_to_one"},{"name":"SALES_TO_PRODUCTS","relationship_type":"many_to_one"},{"name":"SALES_TO_REGIONS","relationship_type":"many_to_one"},{"name":"SALES_TO_REPS","relationship_type":"many_to_one"},{"name":"SALES_TO_VENDORS","relationship_type":"many_to_one"}]}')
	;


-- ========================================================================
-- MARKETING SEMANTIC VIEW
-- ========================================================================
create or replace semantic view EDW.MARKETING_SEMANTIC_VIEW
	tables (
		ACCOUNTS as SF_ACCOUNTS primary key (ACCOUNT_ID) with synonyms=('accounts','clients','customers') comment='Customer account information for revenue analysis',
		CAMPAIGNS as MARKETING_CAMPAIGN_FACT primary key (CAMPAIGN_FACT_ID) with synonyms=('campaign data','marketing campaigns') comment='Marketing campaign performance data',
		CAMPAIGN_DETAILS as CAMPAIGN_DIM primary key (CAMPAIGN_KEY) with synonyms=('campaign details','campaign info') comment='Campaign dimension with objectives and names',
		CHANNELS as CHANNEL_DIM primary key (CHANNEL_KEY) with synonyms=('channels','marketing channels') comment='Marketing channel information',
		CONTACTS as SF_CONTACTS primary key (CONTACT_ID) with synonyms=('contacts','leads','prospects') comment='Contact records generated from marketing campaigns',
		CONTACTS_FOR_OPPORTUNITIES as SF_CONTACTS primary key (CONTACT_ID) with synonyms=('opportunity contacts') comment='Contact records generated from marketing campaigns, specifically for opportunities, not leads',
		OPPORTUNITIES as SF_OPPORTUNITIES primary key (OPPORTUNITY_ID) with synonyms=('deals','opportunities','sales pipeline') comment='Sales opportunities and revenue data',
		PRODUCTS as PRODUCT_DIM primary key (PRODUCT_KEY) with synonyms=('items','products') comment='Product dimension for campaign-specific analysis',
		REGIONS as REGION_DIM primary key (REGION_KEY) with synonyms=('markets','regions','territories') comment='Regional information for campaign analysis'
	)
	relationships (
		CAMPAIGNS_TO_CHANNELS as CAMPAIGNS(CHANNEL_KEY) references CHANNELS(CHANNEL_KEY),
		CAMPAIGNS_TO_DETAILS as CAMPAIGNS(CAMPAIGN_KEY) references CAMPAIGN_DETAILS(CAMPAIGN_KEY),
		CAMPAIGNS_TO_PRODUCTS as CAMPAIGNS(PRODUCT_KEY) references PRODUCTS(PRODUCT_KEY),
		CAMPAIGNS_TO_REGIONS as CAMPAIGNS(REGION_KEY) references REGIONS(REGION_KEY),
		CONTACTS_TO_ACCOUNTS as CONTACTS(ACCOUNT_ID) references ACCOUNTS(ACCOUNT_ID),
		CONTACTS_TO_CAMPAIGNS as CONTACTS(CAMPAIGN_NO) references CAMPAIGNS(CAMPAIGN_FACT_ID),
		CONTACTS_TO_OPPORTUNITIES as CONTACTS_FOR_OPPORTUNITIES(OPPORTUNITY_ID) references OPPORTUNITIES(OPPORTUNITY_ID),
		OPPORTUNITIES_TO_ACCOUNTS as OPPORTUNITIES(ACCOUNT_ID) references ACCOUNTS(ACCOUNT_ID),
		OPPORTUNITIES_TO_CAMPAIGNS as OPPORTUNITIES(CAMPAIGN_ID) references CAMPAIGNS(CAMPAIGN_FACT_ID)
	)
	facts (
		CAMPAIGNS.CAMPAIGN_SPEND as spend comment='Marketing spend in dollars',
		CAMPAIGNS.IMPRESSIONS as IMPRESSIONS comment='Number of impressions',
		CAMPAIGNS.LEADS_GENERATED as LEADS_GENERATED comment='Number of leads generated',
		CONTACTS.CONTACT_RECORD as 1 comment='Count of contacts generated',
		OPPORTUNITIES.OPPORTUNITY_RECORD as 1 comment='Count of opportunities created',
		OPPORTUNITIES.REVENUE as AMOUNT comment='Opportunity revenue in dollars'
	)
	dimensions (
		ACCOUNTS.ACCOUNT_NAME as ACCOUNT_NAME with synonyms=('client name','company','customer name') comment='Name of the customer account',
		ACCOUNTS.ACCOUNT_TYPE as ACCOUNT_TYPE with synonyms=('account category','customer type') comment='Type of customer account',
		ACCOUNTS.ANNUAL_REVENUE as ANNUAL_REVENUE with synonyms=('company revenue','customer revenue') comment='Customer annual revenue',
		ACCOUNTS.EMPLOYEES as EMPLOYEES with synonyms=('company size','employee count') comment='Number of employees at customer',
		ACCOUNTS.INDUSTRY as INDUSTRY with synonyms=('industry','sector') comment='Customer industry',
		CAMPAIGNS.CAMPAIGN_DATE as date with synonyms=('campaign date','date') comment='Date of the campaign activity',
		CAMPAIGN_DETAILS.CAMPAIGN_NAME as CAMPAIGN_NAME with synonyms=('campaign','campaign title') comment='Name of the marketing campaign',
		CAMPAIGN_DETAILS.CAMPAIGN_OBJECTIVE as OBJECTIVE with synonyms=('goal','objective','purpose') comment='Campaign objective',
		CHANNELS.CHANNEL_KEY as CHANNEL_KEY,
		CHANNELS.CHANNEL_NAME as CHANNEL_NAME with synonyms=('channel','marketing channel') comment='Name of the marketing channel',
		CONTACTS.ACCOUNT_ID as ACCOUNT_ID,
		CONTACTS.CAMPAIGN_NO as CAMPAIGN_NO,
		CONTACTS.CONTACT_ID as CONTACT_ID,
		CONTACTS.DEPARTMENT as DEPARTMENT with synonyms=('business unit','department') comment='Contact department',
		CONTACTS.EMAIL as EMAIL with synonyms=('email','email address') comment='Contact email address',
		CONTACTS.FIRST_NAME as FIRST_NAME with synonyms=('contact name','first name') comment='Contact first name',
		CONTACTS.LAST_NAME as LAST_NAME with synonyms=('last name','surname') comment='Contact last name',
		CONTACTS.LEAD_SOURCE as LEAD_SOURCE with synonyms=('lead source','source') comment='How the contact was generated',
		CONTACTS.OPPORTUNITY_ID as OPPORTUNITY_ID,
		CONTACTS.TITLE as TITLE with synonyms=('job title','position') comment='Contact job title',
		OPPORTUNITIES.ACCOUNT_ID as ACCOUNT_ID,
		OPPORTUNITIES.CAMPAIGN_ID as CAMPAIGN_ID with synonyms=('campaign fact id','marketing campaign id') comment='Campaign fact ID that links opportunity to marketing campaign',
		OPPORTUNITIES.CLOSE_DATE as CLOSE_DATE with synonyms=('close date','expected close') comment='Expected or actual close date',
		OPPORTUNITIES.OPPORTUNITY_ID as OPPORTUNITY_ID,
		OPPORTUNITIES.OPPORTUNITY_LEAD_SOURCE as lead_source with synonyms=('deal source','opportunity source') comment='Source of the opportunity',
		OPPORTUNITIES.OPPORTUNITY_NAME as OPPORTUNITY_NAME with synonyms=('deal name','opportunity title') comment='Name of the sales opportunity',
		OPPORTUNITIES.OPPORTUNITY_STAGE as STAGE_NAME comment='Stage name of the opportinity. Closed Won indicates an actual sale with revenue',
		OPPORTUNITIES.OPPORTUNITY_TYPE as TYPE with synonyms=('deal type','opportunity type') comment='Type of opportunity',
		OPPORTUNITIES.SALES_SALE_ID as SALE_ID with synonyms=('invoice no','sales id') comment='Sales_ID for sales_fact table that links this opp to a sales record.',
		PRODUCTS.PRODUCT_CATEGORY as CATEGORY_NAME with synonyms=('category','product category') comment='Category of the product',
		PRODUCTS.PRODUCT_KEY as PRODUCT_KEY,
		PRODUCTS.PRODUCT_NAME as PRODUCT_NAME with synonyms=('item','product','product title') comment='Name of the product being promoted',
		PRODUCTS.PRODUCT_VERTICAL as VERTICAL with synonyms=('industry','vertical') comment='Business vertical of the product',
		REGIONS.REGION_KEY as REGION_KEY,
		REGIONS.REGION_NAME as REGION_NAME with synonyms=('market','region','territory') comment='Name of the region'
	)
	metrics (
		CAMPAIGNS.AVERAGE_SPEND as AVG(spend) comment='Average campaign spend',
		CAMPAIGNS.TOTAL_CAMPAIGNS as COUNT(CAMPAIGN_FACT_ID) comment='Total number of campaign activities',
		CAMPAIGNS.TOTAL_IMPRESSIONS as SUM(impressions) comment='Total impressions across campaigns',
		CAMPAIGNS.TOTAL_LEADS as SUM(leads_generated) comment='Total leads generated from campaigns',
		CAMPAIGNS.TOTAL_SPEND as SUM(spend) comment='Total marketing spend',
		CONTACTS.TOTAL_CONTACTS as COUNT(CONTACTS.contact_record) comment='Total contacts generated from campaigns',
		OPPORTUNITIES.AVERAGE_DEAL_SIZE as AVG(OPPORTUNITIES.revenue) comment='Average opportunity size from marketing',
		OPPORTUNITIES.CLOSED_WON_REVENUE as SUM(CASE WHEN OPPORTUNITIES.opportunity_stage = 'Closed Won' THEN OPPORTUNITIES.revenue ELSE 0 END) comment='Revenue from closed won opportunities',
		OPPORTUNITIES.TOTAL_OPPORTUNITIES as COUNT(OPPORTUNITIES.opportunity_record) comment='Total opportunities from marketing',
		OPPORTUNITIES.TOTAL_REVENUE as SUM(OPPORTUNITIES.revenue) comment='Total revenue from marketing-driven opportunities'
	)
	comment='Enhanced semantic view for marketing campaign analysis with complete revenue attribution and ROI tracking'
	with extension (CA='{"tables":[{"name":"ACCOUNTS","dimensions":[{"name":"ACCOUNT_NAME"},{"name":"ACCOUNT_TYPE","sample_values":["Prospect","Partner","Customer"]},{"name":"ANNUAL_REVENUE"},{"name":"EMPLOYEES"},{"name":"INDUSTRY","sample_values":["Retailer","Manufacturing","Tech"]}]},{"name":"CAMPAIGNS","facts":[{"name":"CAMPAIGN_SPEND"},{"name":"IMPRESSIONS"},{"name":"LEADS_GENERATED"}],"metrics":[{"name":"AVERAGE_SPEND"},{"name":"TOTAL_CAMPAIGNS"},{"name":"TOTAL_IMPRESSIONS"},{"name":"TOTAL_LEADS"},{"name":"TOTAL_SPEND"}],"time_dimensions":[{"name":"CAMPAIGN_DATE"}]},{"name":"CAMPAIGN_DETAILS","dimensions":[{"name":"CAMPAIGN_NAME"},{"name":"CAMPAIGN_OBJECTIVE","sample_values":["Engagement","Branding","Product Launch"]}]},{"name":"CHANNELS","dimensions":[{"name":"CHANNEL_KEY"},{"name":"CHANNEL_NAME"}]},{"name":"CONTACTS","dimensions":[{"name":"ACCOUNT_ID"},{"name":"CAMPAIGN_NO"},{"name":"CONTACT_ID"},{"name":"DEPARTMENT"},{"name":"EMAIL"},{"name":"FIRST_NAME"},{"name":"LAST_NAME"},{"name":"LEAD_SOURCE"},{"name":"OPPORTUNITY_ID"},{"name":"TITLE"}],"facts":[{"name":"CONTACT_RECORD"}],"metrics":[{"name":"TOTAL_CONTACTS"}]},{"name":"CONTACTS_FOR_OPPORTUNITIES"},{"name":"OPPORTUNITIES","dimensions":[{"name":"ACCOUNT_ID"},{"name":"CAMPAIGN_ID"},{"name":"CLOSE_DATE"},{"name":"OPPORTUNITY_ID"},{"name":"OPPORTUNITY_LEAD_SOURCE"},{"name":"OPPORTUNITY_NAME"},{"name":"OPPORTUNITY_STAGE","sample_values":["Closed Won","Perception Analysis","Qualification"]},{"name":"OPPORTUNITY_TYPE"},{"name":"SALES_SALE_ID"}],"facts":[{"name":"OPPORTUNITY_RECORD"},{"name":"REVENUE"}],"metrics":[{"name":"AVERAGE_DEAL_SIZE"},{"name":"CLOSED_WON_REVENUE"},{"name":"TOTAL_OPPORTUNITIES"},{"name":"TOTAL_REVENUE"}]},{"name":"PRODUCTS","dimensions":[{"name":"PRODUCT_CATEGORY"},{"name":"PRODUCT_KEY"},{"name":"PRODUCT_NAME"},{"name":"PRODUCT_VERTICAL"}]},{"name":"REGIONS","dimensions":[{"name":"REGION_KEY"},{"name":"REGION_NAME"}]}],"relationships":[{"name":"CAMPAIGNS_TO_CHANNELS","relationship_type":"many_to_one"},{"name":"CAMPAIGNS_TO_DETAILS","relationship_type":"many_to_one"},{"name":"CAMPAIGNS_TO_PRODUCTS","relationship_type":"many_to_one"},{"name":"CAMPAIGNS_TO_REGIONS","relationship_type":"many_to_one"},{"name":"CONTACTS_TO_ACCOUNTS","relationship_type":"many_to_one"},{"name":"CONTACTS_TO_CAMPAIGNS","relationship_type":"many_to_one"},{"name":"CONTACTS_TO_OPPORTUNITIES","relationship_type":"many_to_one"},{"name":"OPPORTUNITIES_TO_ACCOUNTS","relationship_type":"many_to_one"},{"name":"OPPORTUNITIES_TO_CAMPAIGNS"}],"verified_queries":[{"name":"include opps that turned in to sales deal","question":"include opps that turned in to sales deal","sql":"WITH campaign_impressions AS (\\n  SELECT\\n    c.campaign_key,\\n    cd.campaign_name,\\n    SUM(c.impressions) AS total_impressions\\n  FROM\\n    campaigns AS c\\n    LEFT OUTER JOIN campaign_details AS cd ON c.campaign_key = cd.campaign_key\\n  WHERE\\n    YEAR(c.campaign_date) = 2025\\n  GROUP BY\\n    c.campaign_key,\\n    cd.campaign_name\\n),\\ncampaign_opportunities AS (\\n  SELECT\\n    c.campaign_key,\\n    COUNT(o.opportunity_record) AS total_opportunities,\\n    COUNT(\\n      CASE\\n        WHEN o.opportunity_stage = ''Closed Won'' THEN o.opportunity_record\\n      END\\n    ) AS closed_won_opportunities\\n  FROM\\n    campaigns AS c\\n    LEFT OUTER JOIN opportunities AS o ON c.campaign_fact_id = o.campaign_id\\n  WHERE\\n    YEAR(c.campaign_date) = 2025\\n  GROUP BY\\n    c.campaign_key\\n)\\nSELECT\\n  ci.campaign_name,\\n  ci.total_impressions,\\n  COALESCE(co.total_opportunities, 0) AS total_opportunities,\\n  COALESCE(co.closed_won_opportunities, 0) AS closed_won_opportunities\\nFROM\\n  campaign_impressions AS ci\\n  LEFT JOIN campaign_opportunities AS co ON ci.campaign_key = co.campaign_key\\nORDER BY\\n  ci.total_impressions DESC NULLS LAST","use_as_onboarding_question":false,"verified_by":"Trent Foley","verified_at":1760021805}]}')
	;



-- ========================================================================
-- HR SEMANTIC VIEW
-- ========================================================================
create or replace semantic view EDW.HR_SEMANTIC_VIEW
	tables (
		DEPARTMENTS as DEPARTMENT_DIM primary key (DEPARTMENT_KEY) with synonyms=('business units','departments') comment='Department dimension for organizational analysis',
		EMPLOYEES as EMPLOYEE_DIM primary key (EMPLOYEE_KEY) with synonyms=('employees','staff','workforce') comment='Employee dimension with personal information',
		HR_RECORDS as HR_EMPLOYEE_FACT primary key (HR_FACT_ID) with synonyms=('employee records','hr data') comment='HR employee fact data for workforce analysis',
		JOBS as JOB_DIM primary key (JOB_KEY) with synonyms=('job titles','positions','roles') comment='Job dimension with titles and levels',
		LOCATIONS as LOCATION_DIM primary key (LOCATION_KEY) with synonyms=('locations','offices','sites') comment='Location dimension for geographic analysis'
	)
	relationships (
		HR_TO_DEPARTMENTS as HR_RECORDS(DEPARTMENT_KEY) references DEPARTMENTS(DEPARTMENT_KEY),
		HR_TO_EMPLOYEES as HR_RECORDS(EMPLOYEE_KEY) references EMPLOYEES(EMPLOYEE_KEY),
		HR_TO_JOBS as HR_RECORDS(JOB_KEY) references JOBS(JOB_KEY),
		HR_TO_LOCATIONS as HR_RECORDS(LOCATION_KEY) references LOCATIONS(LOCATION_KEY)
	)
	facts (
		HR_RECORDS.ATTRITION_FLAG as ATTRITION_FLAG with synonyms=('churn_status','employee_departure_flag','employee_exit_indicator','employee_retention_status','separation_flag','turnover_indicator') comment='Attrition flag. value is 0 if employee is currently active. 1 if employee quit & left the company. Always filter by 0 to show active employees unless specified otherwise',
		HR_RECORDS.EMPLOYEE_SALARY as SALARY comment='Employee salary in dollars'
	)
	dimensions (
		DEPARTMENTS.DEPARTMENT_NAME as DEPARTMENT_NAME with synonyms=('business unit','department','division') comment='Name of the department',
		EMPLOYEES.EMPLOYEE_NAME as EMPLOYEE_NAME with synonyms=('director','employee','executive','manager','person','sales rep','staff member') comment='Name of the employee',
		EMPLOYEES.GENDER as GENDER with synonyms=('gender','sex') comment='Employee gender',
		EMPLOYEES.HIRE_DATE as HIRE_DATE with synonyms=('hire date','start date') comment='Date when employee was hired',
		HR_RECORDS.RECORD_DATE as DATE with synonyms=('date','record date') comment='Date of the HR record',
		JOBS.JOB_LEVEL as job_level with synonyms=('grade','level','seniority') comment='Job level or grade',
		JOBS.JOB_TITLE as job_title with synonyms=('job title','position','role') comment='Employee job title',
		LOCATIONS.LOCATION_NAME as location_name with synonyms=('location','office','site') comment='Work location'
	)
	metrics (
		HR_RECORDS.ATTRITION_COUNT as SUM(ATTRITION_FLAG) comment='Number of employees who left',
		HR_RECORDS.AVG_SALARY as AVG(SALARY) comment='average employee salary',
		HR_RECORDS.TOTAL_EMPLOYEES as COUNT(HR_FACT_ID) comment='Total number of employees',
		HR_RECORDS.TOTAL_SALARY_COST as SUM(SALARY) comment='Total salary cost'
	)
	comment='Semantic view for HR analytics and workforce management'
	with extension (CA='{"tables":[{"name":"DEPARTMENTS","dimensions":[{"name":"DEPARTMENT_NAME","sample_values":["Finance","Accounting","Treasury"]}]},{"name":"EMPLOYEES","dimensions":[{"name":"EMPLOYEE_NAME"},{"name":"GENDER","sample_values":["F","M"]}],"time_dimensions":[{"name":"HIRE_DATE"}]},{"name":"HR_RECORDS","facts":[{"name":"ATTRITION_FLAG","sample_values":["0","1"]},{"name":"EMPLOYEE_SALARY"}],"metrics":[{"name":"ATTRITION_COUNT"},{"name":"AVG_SALARY"},{"name":"TOTAL_EMPLOYEES"},{"name":"TOTAL_SALARY_COST"}],"time_dimensions":[{"name":"RECORD_DATE"}]},{"name":"JOBS","dimensions":[{"name":"JOB_LEVEL"},{"name":"JOB_TITLE","sample_values":["Data Analyst","Engineer","HR Manager"]}]},{"name":"LOCATIONS","dimensions":[{"name":"LOCATION_NAME"}]}],"relationships":[{"name":"HR_TO_DEPARTMENTS","relationship_type":"many_to_one"},{"name":"HR_TO_EMPLOYEES","relationship_type":"many_to_one"},{"name":"HR_TO_JOBS","relationship_type":"many_to_one"},{"name":"HR_TO_LOCATIONS","relationship_type":"many_to_one"}],"verified_queries":[{"name":"List of all active employees","question":"List of all active employees","sql":"select\\n  h.employee_key,\\n  e.employee_name,\\nfrom\\n  employees e\\n  left join hr_records h on e.employee_key = h.employee_key\\ngroup by\\n  all\\nhaving\\n  sum(h.attrition_flag) = 0;","use_as_onboarding_question":false,"verified_by":"Nick Akincilar","verified_at":1753846263},{"name":"List of all inactive employees","question":"List of all inactive employees","sql":"SELECT\\n  h.employee_key,\\n  e.employee_name\\nFROM\\n  employees AS e\\n  LEFT JOIN hr_records AS h ON e.employee_key = h.employee_key\\nGROUP BY\\n  ALL\\nHAVING\\n  SUM(h.attrition_flag) > 0","use_as_onboarding_question":false,"verified_by":"Nick Akincilar","verified_at":1753846300}],"custom_instructions":"- Each employee can have multiple hr_employee_fact records. \\n- Only one hr_employee_fact record per employee is valid and that is the one which has the highest date value."}')
	;

-- ========================================================================
-- VERIFICATION
-- ========================================================================

-- Show all semantic views
SHOW SEMANTIC VIEWS;

-- Show dimensions for each semantic view
SHOW SEMANTIC DIMENSIONS;

-- Show metrics for each semantic view
SHOW SEMANTIC METRICS;

## Snowflake Intelligence
Snowflake Intelligence uses agents, which are AI models that are connected to one or more semantic views, semantic models, Cortex search services, and tools. Agents can answer questions, provide insights, and show visualizations. Snowflake Intelligence is powered by Cortex AISQL, Cortex Analyst, and Cortex Search.

> ℹ️ **IMPORTANT**: By default, Snowflake Intelligence uses the default role and the default warehouse of the user. When you invite others to use Snowflake Intelligence, make sure they’ve set a default role and warehouse. All of the queries from Snowflake Intelligence use the user’s credentials. All role-based access control and data-masking policies associated with the user automatically apply to all interactions and conversations with the agent.

### Setup
To set up Snowflake Intelligence for your users, you must configure agent privileges:
1. Create the **snowflake_intelligence** configuration database. This holds the configuration object and the other objects used to support Snowflake Intelligence.
2. Create the **AGENTS** schema to store the agents and make them discoverable to everyone.
3. Grant the **USAGE** privelege on the newly created database and schema to the role that should have access to use Snowflake Intelligence. For this lab, we will grant to the **PUBLIC** role.
4. Grant the **CREATE AGENT** privilege on the **AGENTS** schema to any role that should be able to create agents for Snowflake Intelligence.


In [ ]:
-- Switch to accountadmin role to setup Snowflkae Intelligence within the account
USE ROLE ACCOUNTADMIN;

-- Enable Snowflake Intelligence by creating the Config DB & Schema
CREATE DATABASE IF NOT EXISTS snowflake_intelligence;
CREATE SCHEMA IF NOT EXISTS snowflake_intelligence.agents;

-- Allow anyone to see the agents in this schema
GRANT USAGE ON DATABASE snowflake_intelligence TO ROLE PUBLIC;
GRANT USAGE ON SCHEMA snowflake_intelligence.agents TO ROLE PUBLIC;

GRANT CREATE AGENT ON SCHEMA snowflake_intelligence.agents TO ROLE IDENTIFIER($hol_admin_role_name);

-- Switch back to the HOL admin role
USE ROLE IDENTIFIER($hol_admin_role_name);

### Create an Agent

With the Cortex search services and semantic views we have created, we will now create an agent that uses these tools to orchestrate calls during reasoning and answer curation.

1. In the left-hand navigation menu of Snowsight, select AI & ML » Agents.
2. Select Create agent and enter the following values:
    - Platform integration: select **Create this agent for Snowflake Intelligence**
    - Agent object name: `ENTERPRISE_DATA_AGENT`
    - Display name: `Enterprise Data Agent`

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/18.png)
    
3. Select **Create agent**.
4. After the agent has been created, select the agent from the list of agents and select **Edit**.
5. On the **About** tab, enter the following:
    - Description: `This is an agent that can answer questions about company specific Sales, Marketing, HR & Finance questions.`
    - Example questions: `What are our monthly sales last 12 months?`
    
6. On the **Tools** tab, next to Cortex Analyst select **+ Add** and enter the following:
    - Semantic view: select the `EVOLV_E2E_SNOWFLAKE_AI_HOL` database and `EDW` schema and then select `FINANCE_SEMANTIC_VIEW` from the drop down menu
    - Name: `Finance-Data`
    - Description: select **Generate with Cortex**
    - Warehouse: select **Custom** and then select `EVOLV_AI_HOL_WH` from the drop down menu.
    - Query timeout: `30`
    
7. Select **Add**.
8. Repeat stpes 6-7 for the remaining 3 Cortex Analyst tools using the following tool names:
    
    | Semantic View             | Tool Name        |
    | ------------------------- | ---------------- |
    | `SALES_SEMANTIC_VIEW`     | `Sales-Data`     |
    | `HR_SEMANTIC_VIEW`        | `HR-Data`        |
    | `MARKETING_SEMANTIC_VIEW` | `Marketing-Data` |

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/19.png)

9. Select **Save** in the upper right to save our progress thus far configuring the Agent.
10. Find the Cortex Search Services section, select **+ Add**, and enter the following:
    - Cortex Search: select the `EVOLV_E2E_SNOWFLAKE_AI_HOL` database and `EDW` schema and then select `SEARCH_FINANCE_DOCS` from the drop down menu
    - Max results: `4`
    - ID column: `FILE_URL`
    - Title column: `TITLE`
    - Name: `Finance-Docs`
    - Description: `This tools should be used to search unstructured docs related to the finance department.`
11. Select **Add**.
12. Repeat stpes 10-11 for the remaining 3 Cortex Search tools using the following tool names and descriptions:
    
    | Cortex Search             | Tool Name        | Description                                                                                |
    | ------------------------- | ---------------- | ------------------------------------------------------------------------------------------ |
    | `SEARCH_SALES_DOCS`       | `Sales-Docs`     | This tools should be used to search unstructured docs related to the Sales department.     |
    | `SEARCH_HR_DOCS`          | `HR-Docs`        | This tools should be used to search unstructured docs related to the HR department.        |
    | `SEARCH_MARKETING_DOCS`   | `Marketing-Docs` | This tools should be used to search unstructured docs related to the Marketing department. |

13. Select **Save** in the upper right to save our progress thus far configuring the Agent.

    ![](https://raw.githubusercontent.com/evolvconsulting/snowflake-ai-demo/refs/heads/main/img/20.png)

14. Move to the **Orchestration** tab, and enter the following:
    - Model: `auto`
    - Orchestration instructions:
        ```
        Use Cortex Search for known entities and pass the results to Cortex Analyst for detailed analysis.

        If answering sales related question from Sales-Data, always make sure to include the PRODUCT_CATEGORIES table & filter the product category VERTICAL by 'Retail' for all questions but don't show this fact while explaining thinking steps.

        For Marketing-Data:
        Opportunity Status=Closed_Won indicates an actual sale.
        SalesID in Marketing-Data links an opportunity to a Sales record in Sales -Data SalesID columns.
        ```
    - Response instructions:
        ```
        You are a data analyst who has access to sales, finance, marketing & HR data. If the user does not specify a date range assume it is for year 2025. Leverage data from all domains to analyze & answer user questions. Provide visualizations if possible. Trendlines should default to line charts and categories to bar charts.
        ```

13. Select **Save** in the upper right.

14. Move to the **Access** tab. For this lab, we are not going to make any changes.  In order to make the agent visible for other users, grant USAGE privileges on the agent for particular roles.

    > NOTE: Granting USAGE on the agent **DOES NOT** grant privileges to the underlying data and tools.

15. In addition to configuring the agent with Snowsight, you can view and managed Agents using SQL. Lets view the agent we just created:

In [ ]:
-- Change database context to the SNOWFLAKE_INTELLIGENCE db
USE DATABASE SNOWFLAKE_INTELLIGENCE;

-- List all agents
SHOW AGENTS IN SCHEMA AGENTS;

-- Show the details and agent specification for the ENTERPRISE_DATA_AGENT
DESC AGENT AGENTS.ENTERPRISE_DATA_AGENT;

## Give it a Spin

Now that we have created our agent, let's head over to the Snowflake Intelligence UI and give it a spin. In the left-hand navigation menu of Snowsight, select AI & ML » Snowflake Intelligence.

### Agent Capabilities

The Enterprise Data Agent can:
- **Analyze structured data** across Finance, Sales, Marketing, and HR domains
- **Perform revenue attribution** from marketing campaigns to closed deals via Salesforce CRM integration
- **Search unstructured documents** to provide context and policy information
- **Scrape and analyze web content** from any URL to incorporate external data and insights
- **Generate visualizations** including trend lines, bar charts, and analytics
- **Combine insights** from multiple data sources for comprehensive answers
- **Calculate marketing ROI** and customer acquisition costs across the complete customer journey
- **Understand business context** and provide domain-specific insights

The following questions demonstrate the agent's ability to perform cross-domain analysis, connecting insights across Sales, HR, Marketing, and Finance:

### Sales Performance Analysis
1. **Sales Trends & Performance**  
   "Show me monthly sales trends for 2025 with visualizations. Which months had the highest revenue?"

2. **Top Products & Revenue Drivers**  
   "What are our top 5 products by revenue in 2025? Show me their performance by region."

3. **Sales Rep Performance**  
   "Who are our top performing sales representatives? Show their individual revenue contributions and deal counts."

### HR & Workforce Analysis
1. **Sales Rep Tenure & Performance Correlation**  
   "What is the average tenure of our top sales reps? Is there a correlation between tenure and sales performance?"

2. **Department Staffing & Costs**  
   "Show me employee headcount and average salary by department. Which departments have the highest attrition rates?"

3. **Workforce Distribution & Performance**  
   "How are our employees distributed across locations? What are the performance differences by location?"

### Marketing Campaign Effectiveness & Revenue Attribution
1. **Campaign ROI & Revenue Generation**  
   "Which marketing campaigns generated the most revenue in 2025? Show me marketing ROI and cost per lead by channel."

2. **Complete Funnel Analysis**  
   "Show me the complete marketing funnel from impressions to closed revenue. Which campaigns have the best conversion rates?"

3. **Channel Revenue Performance**  
   "Compare marketing spend to actual closed revenue by channel. Which channels drive the highest value customers?"

### Finance & Cross-Domain Integration
1. **Marketing Attribution & Revenue Analysis**  
   "Show me revenue generated by each marketing channel. What is our true marketing ROI from campaigns to closed deals?"

2. **Customer Acquisition Cost Analysis**  
   "Calculate our customer acquisition cost by marketing channel. Which channels deliver the most profitable customers?"

3. **Vendor Spend & Policy Compliance**  
   "What are our top 5 vendors in the last 5 years? Check our vendor management policy - are we following procurement guidelines for all transactions?"

### Cross-Functional Insights & External Data
**Web Content Analysis Questions**  
1. **Competitive Intelligence**  
   "Analyze the content from [competitor website URL] and compare their product offerings to our product catalog."

2. **Market Research**  
   "Scrape content from [industry report URL] and analyze how it relates to our sales performance and market positioning."

3. **External Data Integration**  
   "Get the latest information from [company news URL] and analyze its potential impact on our sales forecast."
